In [2]:
!pip install geopy

     |████████████████████████████████| 111 kB 2.0 MB/s eta 0:00:01


In [1]:
import requests
import urllib.parse as urlparse
from geopy import distance

In [2]:
class Route:
    def __init__(self, headers={"X-NCP-APIGW-API-KEY-ID": "naver_KEY_ID",
           "X-NCP-APIGW-API-KEY": "naver_KEY_secret"}):
        self.headers = headers
    
    # 각 출발지 -> 위경도 변환
    def addr_to_xy(self):
        # 주소값 입력
        self.d1_name = input("출발지 1을 입력하세요. : ")
        self.d2_name = input("출발지 2를 입력하세요. : ")
        # URL 설정
        self.d1_url = "https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query={}".format(self.d1_name)
        self.d2_url = "https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query={}".format(self.d2_name)
        # 리퀘스트
        self.response1 = requests.get(self.d1_url, headers=self.headers)
        self.response2 = requests.get(self.d2_url, headers=self.headers)
        # JSON 파싱하여 위경도 추출
        self.d1_x = self.response1.json()["addresses"][0]["x"] 
        self.d1_y = self.response1.json()["addresses"][0]["y"]
        self.d2_x = self.response2.json()["addresses"][0]["x"]
        self.d2_y =  self.response2.json()["addresses"][0]["y"]
        return self.d1_x, self.d1_y, self.d2_x, self.d2_y
    
    # 출발지간 위경도 -> 경로값 변환
    def road_path(self):
        self.d1_xy = str(self.d1_x) + "," + str(self.d1_y)
        self.d2_xy = str(self.d2_x) + "," + str(self.d2_y)
        self.d_option = "traoptimal"
        self.d_url = "https://naveropenapi.apigw.ntruss.com/map-direction/v1/driving?start={}&goal={}&option={}".format(self.d1_xy, self.d2_xy, self.d_option)
        self.response = requests.get(self.d_url, headers=self.headers)
        self.path = self.response.json()["route"]["traoptimal"][0]["path"]
        self.avg_path = self.response.json()["route"]["traoptimal"][0]["summary"]["distance"] / len(self.path)
        print("거리 : {}m, ".format(self.response.json()["route"]["traoptimal"][0]["summary"]["distance"]), len(self.path), ", ", self.avg_path)
        return self.path
    
    # 출발지간 대중교통 경로
    def trans_path(self, path_type=0):
        self.path_type = path_type
        self.key = 'ODSAY KEY'
        self.url = 'https://api.odsay.com/v1/api/searchPubTransPathT'
        self.params = {'apiKey' : self.key,
                  'SX' : self.d1_x,
                  'SY' : self.d1_y,
                  'EX' : self.d2_x,
                  'EY' : self.d2_y,
                  'SearchPathType' : self.path_type,
                  'OPT':0,
                 }
        self.url = self.url + '?' + urlparse.urlencode(self.params)
        self.response = requests.get(self.url)
        # 경로내 정류장 위경도 추출
        self.transit_count = self.response.json()['result']['path'][0]['info']['busTransitCount'] + self.response.json()['result']['path'][0]['info']['subwayTransitCount']
        self.distance = round(self.response.json()['result']['path'][0]['info']['totalDistance'] / 1000,2)
        self.stop_lat_lng = []
        for i in range(1, 2*self.transit_count, 2):
            self.stop_info = self.response.json()['result']['path'][0]['subPath'][i]['passStopList']['stations']
            self.stop_lat_lng += [(float(a['y']),float(a['x'])) for a in self.stop_info]
        # 경로가 1km이하거나, 정류장이 5개 미만이면 모든 위경도 표출하고, 이외에는 중간 6개의 정류장 위경도만 표출
        if self.distance <= 1 or len(self.stop_lat_lng) <= 5:
            self.meet_point = self.stop_lat_lng
        else:
            self.meet_point = self.stop_lat_lng[len(self.stop_lat_lng)//2-3:len(self.stop_lat_lng)//2+3]
        print('거리: {}km'.format(self.distance))
        return self.meet_point, self.stop_lat_lng

In [3]:
rte = Route()

In [4]:
d1_x, d1_y, d2_x, d2_y = rte.addr_to_xy()

출발지 1을 입력하세요. : 서울시 강서구 강서로 426
출발지 2를 입력하세요. : 서울시 도봉구 방학2동 344-5


In [5]:
d1_x, d1_y, d2_x, d2_y

('126.8406143', '37.5633724', '127.0303294', '37.6672368')

In [6]:
# 대중교통 이동
meet_points, all_points =rte.trans_path()

거리: 37.99km


In [7]:
# 경로내 위, 경도 최대 최소값 구하기
lat_list = [point[0] for point in all_points]
lng_list = [point[1] for point in all_points]

In [8]:
min(lat_list), max(lat_list), min(lng_list), max(lng_list)

(37.542432, 37.667715, 126.827282, 127.038225)

In [10]:
df = pd.read_csv('mango_r.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7830 entries, 0 to 7829
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   name          7830 non-null   object 
 1   tel           7566 non-null   object 
 2   addr          7830 non-null   object 
 3   cusine_code   7830 non-null   int64  
 4   rating        7830 non-null   float64
 5   review_count  7830 non-null   int64  
 6   lat           7830 non-null   float64
 7   lng           7830 non-null   float64
 8   thumbnail     7830 non-null   object 
dtypes: float64(3), int64(2), object(4)
memory usage: 550.7+ KB


In [11]:
df['latlng'] = df.apply(lambda x:(x['lat'], x['lng']), axis=1)
df.tail()

,name,tel,addr,cusine_code,rating,review_count,lat,lng,thumbnail,latlng
7825,박종근과자점,02-599-4295,서울시 서초구 반포동 1048,7,3.9,24,37.501415,126.988873,https://mp-seoul-image-production-s3.mangoplat...,"(37.501415, 126.9888727)"
7826,소운빙,02-537-1195,서울시 서초구 반포동 978,7,3.9,22,37.501787,126.990124,https://mp-seoul-image-production-s3.mangoplat...,"(37.5017873, 126.9901237)"
7827,스마일포차,02-534-0236,서울시 서초구 반포동 938,8,3.6,21,37.502126,126.990072,https://mp-seoul-image-production-s3.mangoplat...,"(37.50212641, 126.990072)"
7828,에이치플러스와이,02-3477-2612,서울시 서초구 반포동 809 반포아파트,7,0.0,14,37.503467,126.988296,https://mp-seoul-image-production-s3.mangoplat...,"(37.5034672, 126.9882958)"
7829,돈거돈락,02-534-9290,서울시 서초구 반포동 1053 204호,1,0.0,7,37.501167,126.987020,https://mp-seoul-image-production-s3.mangoplat...,"(37.50116732, 126.98701952)"


In [12]:
# 경로 square 내 맛집 추출
direction_square = df[(df['lat']<=max(lat_list)) & (df['lat']>=min(lat_list)) & (df['lng']>=min(lng_list)) & (df['lng']<=max(lng_list))]

In [13]:
# square 내 경로 1km 이내 맛집 추출
matzip=pd.DataFrame()
for lat_lng in direction_square['latlng']:
    for point in meet_points:
        dis = distance.distance(point, lat_lng).km
        if dis <= 1:
            df_1 = direction_square[direction_square['latlng'] == lat_lng]
            matzip = matzip.append(df_1).reset_index(drop=True)

In [14]:
matzip

,name,tel,addr,cusine_code,rating,review_count,lat,lng,thumbnail,latlng
0,슬로우브레드파파,02-923-5218,서울시 성북구 동선동2가 142-2,7,4.1,9,37.590676,127.020244,https://mp-seoul-image-production-s3.mangoplat...,"(37.590676, 127.020244)"
1,왕십리순대국,02-3672-1133,서울시 종로구 이화동 207-2,1,0.0,5,37.575949,127.003344,https://mp-seoul-image-production-s3.mangoplat...,"(37.5759485, 127.0033439)"
2,왕십리순대국,02-3672-1133,서울시 종로구 이화동 207-2,1,0.0,5,37.575949,127.003344,https://mp-seoul-image-production-s3.mangoplat...,"(37.5759485, 127.0033439)"
3,오모테산도,02-926-5550,서울시 성북구 보문동1가 208,7,0.0,4,37.585488,127.020279,https://mp-seoul-image-production-s3.mangoplat...,"(37.5854884, 127.0202791)"
4,애정훠궈,010-8280-0868,서울시 성북구 동선동1가 3-3 2F,2,3.9,11,37.592028,127.017461,https://mp-seoul-image-production-s3.mangoplat...,"(37.5920284, 127.0174611)"
...,...,...,...,...,...,...,...,...,...,...
331,창신육회,02-2266-6727,서울시 종로구 종로4가 165-3,1,4.3,140,37.570475,126.999848,https://mp-seoul-image-production-s3.mangoplat...,"(37.57047539999999, 126.9998481)"
332,나누미떡볶이,02-747-0881,서울시 종로구 명륜2가 225,1,4.1,237,37.583738,126.997885,https://mp-seoul-image-production-s3.mangoplat...,"(37.5837376, 126.9978852)"
333,나누미떡볶이,02-747-0881,서울시 종로구 명륜2가 225,1,4.1,237,37.583738,126.997885,https://mp-seoul-image-production-s3.mangoplat...,"(37.5837376, 126.9978852)"
334,버거파크,02-955-3005,서울시 성북구 동선동2가 10,4,4.2,124,37.590577,127.018411,https://mp-seoul-image-production-s3.mangoplat...,"(37.59057670000001, 127.0184106)"
